<span style="font-size:1.5em;"><b>Análisis de bases de datos  
Universidad Nacional de Colombia  
2024-2</b></span>
# <b>Siniestralidad Vial en Bogotá: Análisis de Datos para la Gestión de la Movilidad Segura</b>  

<span style="font-size:0.9em;">Laura Nicole Bermudez Santa - labermudezs@unal.edu.co  
David Sebastián Mendoza Cruz - damendozac@unal.edu.co  
David Alejandro Alquichire Rincón - dalquichire@unal.edu.co  
Laura Nicole Bermudez Santa - labermudezs@unal.edu.co  
Juan David Bernal Vesga - jubernalv@unal.edu.co</span>

----

La seguridad vial es un tema de vital importancia para la planificación urbana y la protección de la vida en las ciudades. En Bogotá, la Secretaría Distrital de Movilidad (SDM) ha recopilado datos detallados sobre siniestralidad vial desde el año 2007, los cuales incluyen información sobre lesionados, accidentes, causas, tipos de vehículos involucrados y actores viales. Estos datos, disponibles en la plataforma Enterprise de la SDM, representan una oportunidad única para analizar y comprender los patrones de accidentalidad en la ciudad, con el fin de contribuir a la toma de decisiones informadas en materia de seguridad vial.

Este proyecto tiene los siguientes objetivos:
1. Analizar y clasificar los accidentes de tránsito registrados en Bogotá en la plataforma de la secretaria de movilidad desde el 2007, enfocándose en la relación entre tipo de vehículo, actores viales involucrados, localidades afectadas, el estado de los incidentes, etc. Con el fin de identificar patrones clave y proporcionar insumos para la toma de decisiones en materia de seguridad vial.  
2. Proporcionar herramientas visuales que guíen la toma de decisiones en materia de seguridad vial en resultado del análisis estadístico realizado.
3. Determinar tendencias y generar pronósticos que permitan anticipar posibles escenarios de siniestralidad en el futuro, mediante una proyección temporal del comportamiento de los accidentes de tránsito, considerando variables como la frecuencia de muertes y accidentes simples.
Buscamos lo que se expone en los objetivos, pero la idea es usar datos abiertos expuestos en la página de la secretaria de movilidad donde tenemos acceso a los siguientes datos de siniestralidad:
Lesionado, accidentes, causa, vehiculo y actor.



----

#### **Manipulación y limpieza de las bases de datos individuales**

La Secretaria de movilidad dispone una base de [datos abiertos](https://datos.movilidadbogota.gov.co/maps/ea243e7de8e846c8bd27e47c08771d66/about) amplia en materia de siniestrialidad en Bogotá la cual ha sido alimentada con datos desde el 2007. En esta sección depuraremos los siguientes data sets:
- __*ACCIDENTE*__: nos brinda información detallada de la cantidad de accidentes que se han registrado en la ciudad de bogotá incluyendo sus coordenadas, la clase de choque, fecha y gravedad. (En este dataset se concentra la mayoria de los datos y es de nuestro interes analizarlo a profundidad).
- __*LESIONADO*__: incluye información de las personas involucrada tales como su condición, genero y edad son los datos mas relevantes.
- __*ACC_VIA*__: caracteristicas relevantes de cada via donde sucedio el accidente, resulta relevante al relacionar el estado de las vías en la ciudad de Bogotá como posible causante de siniestros en ciertos lugares o localidades.
- __*ACC_VEHICULO*__: información general del tipo de vehiculo y el uso de este ya sea publico o particular.
- __*ACC_CAUSA*__: incluye una breve descripción de una posible causa asociada al siniestro.
- __*ACC_ACTOR*__: nos brinda información relevante de la condición del actor causante del accidente.